# Lista 3
---

**Livro:** Applied Mathematical Programming - MIT

**Capítulo:** Integer Programming

**Professor:** Luciano Guimarães de Castro 

### Lucas Machado Moschen
---

In [1]:
using Pkg
pkg"add Debugger"
pkg"activate ../."
pkg"instantiate"
using Plots, LinearAlgebra, JuMP, GLPK, Debugger

  Activating environment at `~/Documents/linear-programming/homeworks/Project.toml`


**2. A company wishes to put together an academic ‘‘package’’ for an executive training program. There are five area colleges, each offering courses in the six fields that the program is designed to touch upon.**

- **The package consists of 10 courses; each of the six fields must be covered.**

- **The tuition (basic charge), assessed when at least one course is taken, at college $i$ is $T_i$ (independent of the number of courses taken).**

- **Each college imposes an additional charge (covering course materials, instructional aids, and so forth) for each course, the charge depending on the college and the field of instructions.**

**Formulate an integer program that will provide the company with the minimum amount it must spend to meet the requirements of the program.**

Inicialmente, vamos definir as variáveis envolvidas. 

Seja $x_{ij}$ o número de cursos do campo $j \in \{1,\dots,6\}$ no colégio $i \in \{1,\dots,5\}$ e $y_i$ uma variável indicadora se a escola $i$ foi utilizada para ter pelo menos um curso. 

Vamos definir $c_{ij}$ o custo adicional para cada curso que depende do colégio $i \in \{1,\dots,5\}$ e do campo $j \in \{1,\dots,6\}$. 

O custo da mensalidade é dado por  $\sum_{i=1}^5 y_i T_i$ e o custo adicional dos cursos utilizados será $\sum_{i=1}^5 \sum_{j=1}^6 c_{ij} x_{ij}$. Portanto, queremos minimizar 

$$
\sum_{i=1}^5 y_i T_i + \sum_{i=1}^5 \sum_{j=1}^6 c_{ij} x_{ij}
$$

Vamos às restrições: 

- Queremos oferecer 10 cursos no total. Portanto, $\sum_{i=1}^5 \sum_{j=1}^6 x_{ij} = 10$. 

- Queremos que cada campo seja coberto em pelo menos um curso, isto é, $\sum_{i=1}^5 x_{ij} \ge 1, j = 1,\dots,6$. 

- Note que $y_i = 0$ se, e somente se, $x_{ij} = 0$. Portanto, coloco a restrição: $y_i \le x_{ij} \le 10y_i$. Assim, $y_i = 0 \implies x_{ij} = 0$ e $y_i = 1 \implies 1 \le x_i \le 10$, como gostaríamos. 

In [2]:
function package(c, T, printing)
    
    model = Model(GLPK.Optimizer);
    
    @variable(model, x[i=1:5, j=1:6], Int);
    @variable(model, y[i=1:5], Bin);
    
    @objective(model, Min, dot(T,y) + sum(c.*x));
    
    @constraint(model, sum(x) == 10);
    @constraint(model, [sum(x[:,j]) for j=1:6] .>= 1);
    for j=1:6
        @constraint(model, [x[i,j] for i=1:5] .>= y)
        @constraint(model, [x[i,j] for i=1:5] .<= 10*y)
    end
    
    if printing
        display(model)
    end
    
    optimize!(model)
    
    return value.(x), value.(y)
end;

Com a função definida, podemos fazer experimentos. Para printar o modelo, troque `false` por `true`. 

In [3]:
c = [ 
    1 1 1 1 1 1;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
];

T = [100;100;100;100;100];

xx, yy = package(c,T, false);

Observe que com todos os custos igualitários, o melhor é escolher uma só faculdade e distribuir os cursos nela de forma que todo campo tenha pelo menos 1 curso oferecido. 

In [4]:
xx

5×6 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  5.0  1.0

In [5]:
yy

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 1.0

No experimento seguinte, observe que temos 5 faculdades iguais, mas uma que só cobra por curso. De fato ela acaba sendo a preferida, por que escolher outra faculdade traria gastos de 100 + o número de cursos, enquanto essa, o gasto será 100. 

In [6]:
c = [ 
    10 10 10 10 10 10;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
    1 1 1 1 1 1;
];

T = [0;100;100;100;100];

xx, yy = package(c,T, false);

In [7]:
xx

5×6 Matrix{Float64}:
 1.0  1.0  1.0  5.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

Por fim, vamos verificar se há divisão quando existem dois campos bem diferentes em facultadades diferentes. Note que essa escolha de parâmetros é bem sensível. Diminuindo em uma unidade um dos cursos na faculdade 5, faria automaticamente a troca de universidade. 

In [8]:
c = [ 
    10 1 1  5 1 1;
    10 1 1 10 1 1;
    10 1 1 10 1 1;
    10 1 1 10 1 1;
    5  1 1 10 1 1;
];

T = [50;50;50;50;50];

xx, yy = package(c,T, false);

xx

5×6 Matrix{Float64}:
 1.0  1.0  1.0  1.0  5.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

**5. Consider the problem:**

$$\text{Maximize } z = x_1 + 2x_2, \text{subject to: } $$

$$x_1 + x_2 \le 8$$
$$-x_1 + x_2 \le 2$$
$$x_1 - x_2 \le 4$$
$$x_2 \ge 0, x_2 \in \mathbb{Z}$$
$$x_1 = 0, 1, 4, \text{ or }6.$$

**a) Reformulate the problem as an equivalent integer linear program.**



Vou considerar que a forma padrão de um [Programa Linear Inteiro](https://en.wikipedia.org/wiki/Integer_programming#Canonical_and_standard_form_for_ILPs) é a seguinte:
$$
\begin{gather}
\text{max } c^Tx \\ 
\text{s.a. } Ax \le b, \\
x \ge 0, \\
x \in \mathbb{Z}^n
\end{gather}
$$

Nesse caso, introduzimos as variáveis binárias $y_0, y_1, y_4, y_6$ em que $y_i = 1 \iff x_1 = i$. 

Com isso, se $\mathcal{I} = \{0,1,4,6\}$, podemos restringir que $x_1 = \sum_{i\in\mathcal{I}} y_i i$ e que $\sum_{i\in\mathcal{I}} y_i = 1$. Para transformar uma igualdade em desigualdade, basta considerar as duas desigualdades. Observe que a restrição $0 \le y_i \le 1$ já é satisfeita dado que $y_0 + y_1 + y_4 + y_6 = 1$

Vamos então, definir o problema de forma geral. Defina $x = (x_1, x_2, y_0, y_1, y_4, y_6)$. 

Se $c = (1,2,0,0,0,0)$, então queremos maximizar $c^Tx$ sujeito a 

$$A = \begin{bmatrix}
 1 & 1 & 0 & 0 & 0 & 0 \\
-1 & 1 & 0 & 0 & 0 & 0 \\
 1 &-1 & 0 & 0 & 0 & 0 \\
 1 & 0 & 0 &-1 &-4 &-6 \\
-1 & 0 & 0 & 1 & 4 & 6 \\
 0 & 0 & 1 & 1 & 1 & 1 \\
 0 & 0 &-1 &-1 &-1 &-1 \\
\end{bmatrix}\begin{bmatrix}
x_1 \\ x_2 \\ y_0 \\ y_1 \\ y_4 \\ y_6
\end{bmatrix} \le b = \begin{bmatrix}
8 \\ 2 \\ 4 \\ 0 \\ 0 \\ 1 \\ -1 
\end{bmatrix}
$$

E as restrições $x \ge 0$ e $x \in \mathbb{Z}^6$. 

**b) How would your answer to part (a) change if the objective function were changed to:**

$$\text{Maximize }z = x_1^2 + 2x_2$$

Nesse caso basta definir $x_1' = x_1^2$ que pode assumir os valores $0,1,16,36$ (que deve ser incorporado nas desigualdades como explicado no item anterior.  

In [9]:
# Definindo parâmetros 
c = [1 2 0 0 0 0]

A = [ 1  1  0  0  0  0;
     -1  1  0  0  0  0;
      1 -1  0  0  0  0;
      1  0  0 -1 -4 -6;
     -1  0  0  1  4  6;
      0  0  1  1  1  1;
      0  0 -1 -1 -1 -1]

b = [8;2;4;0;0;1;-1]; 

Por simplificação vamos usar que $x_3 = y_0, x_4 = y_1, x_5 = y_4, x_6 = y_6$. 

In [10]:
model5 = Model(GLPK.Optimizer)

@variable(model5, x[i=1:6] >= 0, Int)

@objective(model5, Max, c ⋅ x)

@constraint(model5, A*x .<= b)

model5

A JuMP Model
Maximization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [11]:
optimize!(model5)

value.(x)

6-element Vector{Float64}:
 4.0
 4.0
 0.0
 0.0
 1.0
 0.0

Obtemos que a solução é $x_1 = 4$ e $x_2 = 4$. Observe que $x_5 = y_4 = 1$ como gostaríamos. Podemos visualizar esse exemplo graficamente:

![](exer5.png)

**11. Solve the following integer program using the branch-and-bound technique:**

$$
\begin{align}
    \text{Minimize } &z = 2x_1 - 3x_2 - 4x_3, \\
    \text{subject to:}&  \\
    &-x_1 + x_2+ 3x_3 \le 8, 3x_1 +2x_2 - x_3 \le 10, \\
    &x_1, x_2, x_3 \ge 0 \text{ and integer.}
\end{align}
$$

Para esse problema, a ideia é construir o algoritmo branch-and-bound:

In [12]:
function branch_and_bound(model, index_var_integers)
    optimize!(model)
    xx = yy = zz = value.(x)
    obj = obj1 = obj2 = objective_value(model)
    print("The estimated was: ")
    print(xx)
    print("\n")
    for i=1:length(index_var_integers)
        if ~(floor(xx[i]) == ceil(xx[i]))
            enter = 1
            model1 = model 
            @constraint(model1, x[i] <= floor(xx[i]))
            yy, obj1 = branch_and_bound(model1, index_var_integers)
            model2 = model 
            @constraint(model2, x[i] >= ceil(xx[i]))
            zz, obj2 = branch_and_bound(model2, index_var_integers)
        end
    end
    xx = (obj1 < obj2 ? zz : yy)
    obj = (obj1 < obj2 ? obj2 : obj1)
    return xx, obj
    end;

Definimos o modelo sem a restrição de variável inteira. 

In [13]:
bb_model = Model(GLPK.Optimizer)

@variable(bb_model, x[i=1:3] >= 0);
@objective(bb_model, Max, -dot([2 -3 -4], x))
@constraint(bb_model, dot([-1 1 3], x) <= 8)
@constraint(bb_model, dot([3 2 -1], x) <= 1)
bb_model

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [14]:
branch_and_bound(bb_model, [1 2 3])

The estimated was: [0.0, 1.5714285714285712, 2.142857142857143]
The estimated was: [0.0, 1.0, 2.333333333333333]
The estimated was: [0.0, 1.0, 2.0]
The estimated was: [0.0, 1.0, 2.0]
The estimated was: [0.0, 1.0, 2.0]
The estimated was: [0.0, 1.0, 2.0]
The estimated was: [0.0, 1.0, 2.0]


([0.0, 1.0, 2.0], 11.0)

Vamos comparar essa solução com o algoritmo do Julia. 

In [15]:
bb_model2 = Model(GLPK.Optimizer)

@variable(bb_model2, x[i=1:3] >= 0, Int);
@objective(bb_model2, Max, -dot([2 -3 -4], x))
@constraint(bb_model2, dot([-1 1 3], x) <= 8)
@constraint(bb_model2, dot([3 2 -1], x) <= 1)

optimize!(bb_model2)
value.(x)

3-element Vector{Float64}:
 0.0
 1.0
 2.0

As soluções são as mesmas! Então nosso algoritmo acertou!